In [1]:
import sys
from pathlib import Path
import joblib
import json

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import load_dataset, Counterfactuals

In [2]:
X, y = load_dataset("california_housing", preprocess=True, include_description=True)

{'AveBedrms': 'Average number of bedrooms per household',
 'AveOccup': 'Average house occupancy (household size)',
 'AveRooms': 'Average number of rooms per household',
 'HouseAge': 'Median house age in block group',
 'Latitude': 'Block group latitude',
 'Longitude': 'Block group longitude',
 'MedInc': 'Median income in block group',
 'Population': 'Block group population',
 'target': 'MedHouseVal, Median house value in 100k USD'}


In [3]:
ARTIFACT_DIR = Path("../artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

model = joblib.load(ARTIFACT_DIR / "xgboost_regressor.pkl")

In [4]:
instance = X.iloc[0:1].copy()
instance_outcome = y.iloc[0:1]
instance.loc[:, 'outcome'] = instance_outcome.values

In [5]:
instance

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23,4.526


In [6]:
cf = Counterfactuals(X=X, y=y, model=model)

In [7]:
genetic = cf.get_counterfactuals(instance, 5, "genetic", lower_limit=1.6, upper_limit=2)

Generation 6; counterfactuals found: 10/5


In [11]:
prototypes = cf.get_counterfactuals(instance, 5, "prototypes", lower_limit=1.6, upper_limit=2)

In [9]:
genetic

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,2.254448,41.000000,6.984127,1.02381,322.000000,2.555556,37.880000,-122.230000,1.961538
1,4.978753,37.846316,6.651624,1.02381,1923.844071,12.865296,45.792134,-162.284905,1.805433
2,1.050407,41.000000,6.984127,1.02381,322.000000,2.555556,37.880000,-122.230000,1.737393
3,4.687804,41.000000,6.984127,1.02381,322.000000,2.555556,69.517539,-122.230000,1.631226
4,1.164060,41.000000,6.984127,1.02381,322.000000,2.555556,37.880000,-122.230000,1.746060


In [12]:
prototypes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
19529,7.3841,40.0,6.698413,1.000000,158.0,2.507937,37.65,-120.98,1.722
357,6.0749,39.0,6.751852,1.059259,763.0,2.825926,37.76,-122.15,1.969
1370,5.1265,40.0,6.498195,1.046931,761.0,2.747292,38.02,-122.16,1.961
791,5.7309,36.0,4.938053,0.880531,662.0,2.929204,37.64,-122.08,1.779
1404,5.0445,37.0,6.273196,1.025773,552.0,2.845361,37.96,-122.07,1.962
